### Writing a custom RNN cell

Starting with a GRU cell, instead of taking a single transformation of its input, we enable it to take a weighted average of multiple transformations of its input. That is, using the notation from Cho et al. (2014), instead of using $Wx$ in our candidate state, $~h(t)=tanh(Wx+U(r⊙h^{(t−1)})$, we use a weighted average of $W_1x$, $W_2x…W_nx$ for some $n$. In other words, we will replace $Wx$ with $Σλ_iW_ix$ for some weights $λ_i$ that sum to 1. The vector of weights, $λ$, will be calculated as $λ=softmax(W_{avg}x^{(t)}+U_{avg}h^{(t−1)}+b)$. The idea is that we might benefit from treat the input differently in different scenarios (e.g., we may want to treat verbs differently than nouns). To write the custom cell, we need to extend `tf.nn.rnn_cell.RNNCell`.

In [2]:
import tensorflow as tf

/Users/mzielinski/anaconda3/envs/keras/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)
/Users/mzielinski/anaconda3/envs/keras/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


First, let’s start with a GRU cell, adapted from Tensorflow’s implementation:

In [3]:
class GRUCell(tf.nn.rnn_cell.RNNCell):
    """Gated Recurrent Unit cell (cf. http://arxiv.org/abs/1406.1078)."""

    def __init__(self, num_units):
        self._num_units = num_units

    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__):  # "GRUCell"
            with tf.variable_scope("Gates"):  # Reset gate and update gate.
                # We start with bias of 1.0 to not reset and not update.
                ru = tf.nn.rnn_cell._linear([inputs, state],
                                        2 * self._num_units, True, 1.0)
                ru = tf.nn.sigmoid(ru)
                r, u = tf.split(1, 2, ru)
            with tf.variable_scope("Candidate"):
                c = tf.nn.tanh(tf.nn.rnn_cell._linear([inputs, r * state],
                                             self._num_units, True))
            new_h = u * state + (1 - u) * c
        return new_h, new_h

Extending RNNCell means overriding at least the `state_size` property, the `output_size` property, and the `call` method. Tensorflow’s prebuilt cells internally represent state either as a signal tensor or as a tuple of tensors. 
* If it is a **single tensor**, it gets broken down into cell and hidden states (or whatever the case may be) upon entry into the cell, and then the new states are stuck back together at the end.

* If it's the cell state **as a tuple**. In this case, the `state_size` property is just a tuple with the lengths of whichever states you’re keeping track of. `output_size` is the length of the output of the cell. The `call` function is where the logic of your cell will live. RNNCell’s `__call__` method will wrap your call method and help with scoping and other logistics.

In order for your subclass to be a valid RNNCell, the `call` method must accept parameters `input` and `state`, and return a tuple of `output`, `new_state`, where `state` and `new_state` must have the same form.

Modify the Candidate variable scope of the `__call__` method to do a weighted average as shown below (note that all of the  $W_i$ matrices are created as a single variable and then split into multiple tensors):

In [13]:
class CustomGRUCell(tf.nn.rnn_cell.RNNCell):
    """Gated Recurrent Unit cell (cf. http://arxiv.org/abs/1406.1078)."""

    def __init__(self, num_units, num_weights):
        self._num_units = num_units
        self._num_weights = num_weights # number of transormation matrices

    @property
    def state_size(self):
        return self._num_units

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        with tf.variable_scope(scope or type(self).__name__):  # "CustomGRUCell"
            with tf.variable_scope("Gates"):  # Reset gate and update gate.
                # We start with bias of 1.0 to not reset and not update.
                ru = tf.nn.rnn_cell._linear([inputs, state],
                                        2 * self._num_units, True, 1.0)
                # ^^ handles matrix multiplication and addition of weights and biases.
                ru = tf.nn.sigmoid(ru)
                r, u = tf.split(1, 2, ru)
            with tf.variable_scope("Candidate"):
                lambdas = tf.nn.rnn_cell._linear([inputs, state], self._num_weights, True)
                lambdas = tf.split(1, self._num_weights, tf.nn.softmax(lambdas))

                Ws = tf.get_variable("Ws",
                        shape = [self._num_weights, inputs.get_shape()[1], self._num_units])
                Ws = [tf.squeeze(i) for i in tf.split(0, self._num_weights, Ws)]

                candidate_inputs = []

                for idx, W in enumerate(Ws):
                    candidate_inputs.append(tf.matmul(inputs, W) * lambdas[idx])

                Wx = tf.add_n(candidate_inputs)

                c = tf.nn.tanh(Wx + tf.nn.rnn_cell._linear([r * state],
                                            self._num_units, True, scope="second"))
            new_h = u * stat

## Custom MultiCell

If you’re building a multi-layered RNN where the layers don’t simply pass their output up from layer to layer, you’ll have to build your own version of a `MultiCell`. Much like the built in `MultiRNNCell`, your multicell should extend `RNNCell`.

In this case the cell `state` will be a list, where each element is the cell `state` at the layer corresponding to its index.

Writing your own multicell is useful in two cases. 
* You want to do something to the result of one layer before you pass it into the cell at the next layer, but you don’t want to execute that operation for the lowest layer (otherwise you could just build it into the cell).
* If there’s information from the previous time step that you need to distribute among the different layers, but that doesn’t fit neatly into the paradigm of passing along state from one time step to the next.

For example, in the hierarchical multiscall LSTM, each cell expects to receive the hidden state from the layer above it at the previous time step as part of its input. This doesn’t neatly fit the standard idea of stacked RNNs, so we can’t use the usual MultiRNNCell. For reference, here is the implementation of the <a href="https://github.com/n-s-f/hierarchical-rnn/blob/master/hmlstm/multi_hmlstm_cell.py">MultiHMLSTMCell</a>

## Adding Dropout

Dropout belongs **in between layers**, not on the state or in intra-cell connections. See Zaremba et al. (2015), Recurrent Neural Network Regularization (*“The main idea is to apply the dropout operator only to the non-recurrent connections.”*)

Thus, to apply dropout, we need to wrap the input and/or output of each cell. But what happens when we use MultiRNNCell? How can we have dropout in between layers like in Zaremba et al. (2015)? The answer is to wrap our base RNN cell with dropout, thereby including it as part of the base cell, similar to how we wrapped our three RNN cells into a single MultiRNNCell above. Tensorflow allows us to do this without writing a new RNNCell by using `tf.nn.rnn_cell.DropoutWrapper`:

In [9]:
state_size = 5
num_layers = 3
input_dropout = 0.5
output_dropout = 0.5

cell = tf.nn.rnn_cell.LSTMCell(state_size, state_is_tuple=True)
cell = tf.nn.rnn_cell.DropoutWrapper(cell, input_keep_prob=input_dropout, output_keep_prob=output_dropout)
cell = tf.nn.rnn_cell.MultiRNNCell([cell] * num_layers, state_is_tuple=True)

## Layer normalization

Layer normalization is applied as follows: the initial layer normalization function is applied individually to each training example so as to normalize the output vector of a linear transformation to have a mean of 0 and a variance of 1. In math: $LN_{initial}:v↦\frac{v−μ_v}{\sqrt{σ_v^2}+ϵ}$ for some vector v and some small value of $ϵ$ for numerical stability. For some the same reasons we add scale and shift parameters to the initial batch normalization transform (see my batch normalization post for details), we add scale, $α$, and shift, $β$, parameters here as well, so that the final layer normalization function is: $LN:v↦α⊙\frac{v−μ_v}{\sqrt{σ_v^2}+ϵ}+β$

To add layer normalization to our network, we first write a function that will layer normalization a 2D tensor along its second dimension:

In [10]:
def ln(tensor, scope = None, epsilon = 1e-5):
    """ Layer normalizes a 2D tensor along its second axis """
    assert(len(tensor.get_shape()) == 2)
    m, v = tf.nn.moments(tensor, [1], keep_dims=True)
    if not isinstance(scope, str):
        scope = ''
    with tf.variable_scope(scope + 'layer_norm'):
        scale = tf.get_variable('scale',
                                shape=[tensor.get_shape()[1]],
                                initializer=tf.constant_initializer(1))
        shift = tf.get_variable('shift',
                                shape=[tensor.get_shape()[1]],
                                initializer=tf.constant_initializer(0))
    LN_initial = (tensor - m) / tf.sqrt(v + epsilon)

    return LN_initial * scale + shift

Lei Ba et al. **apply layer normalization to the output of each gate** inside the LSTM cell, which means that we get to take a second shot at writing a new type of RNN cell. We’ll start with Tensorflow’s official code, located here, and modify it accordingly:

In [11]:
class LayerNormalizedLSTMCell(tf.nn.rnn_cell.RNNCell):
    """
    Adapted from TF's BasicLSTMCell to use Layer Normalization.
    Note that state_is_tuple is always True.
    """

    def __init__(self, num_units, forget_bias=1.0, activation=tf.nn.tanh):
        self._num_units = num_units
        self._forget_bias = forget_bias
        self._activation = activation

    @property
    def state_size(self):
        return tf.nn.rnn_cell.LSTMStateTuple(self._num_units, self._num_units)

    @property
    def output_size(self):
        return self._num_units

    def __call__(self, inputs, state, scope=None):
        """Long short-term memory cell (LSTM)."""
        with tf.variable_scope(scope or type(self).__name__):
            c, h = state

            # change bias argument to False since LN will add bias via shift
            concat = tf.nn.rnn_cell._linear([inputs, h], 4 * self._num_units, False)

            i, j, f, o = tf.split(1, 4, concat)

            # add layer normalization to each gate
            i = ln(i, scope = 'i/')
            j = ln(j, scope = 'j/')
            f = ln(f, scope = 'f/')
            o = ln(o, scope = 'o/')

            new_c = (c * tf.nn.sigmoid(f + self._forget_bias) + tf.nn.sigmoid(i) *
                   self._activation(j))

            # add layer_normalization in calculation of new hidden state
            new_h = self._activation(ln(new_c, scope = 'new_h/')) * tf.nn.sigmoid(o)
            new_state = tf.nn.rnn_cell.LSTMStateTuple(new_c, new_h)

            return new_h, new_state

## Generating text